# Rede Neural - MLP 

In [50]:
import torch
from torch import nn
from d2l import torch as d2l
import matplotlib.pyplot as plt
import random

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [55]:
from torchvision import transforms
import torchvision

def carrega_data_fashion_mnist(batch_size, resize=(112, 112)):
    """Download the Fashion-MNIST dataset and then load it into memory.

    Defined in :numref:`sec_utils`"""
    trans = []

    if resize:
        trans.append(transforms.Resize(resize))

    trans.extend([
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])

    trans = transforms.Compose(trans)

    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    
    return (torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True,
                                        num_workers=6),
            torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=False,
                                        num_workers=6))

In [58]:
dados_treino1, dados_teste1 = carrega_data_fashion_mnist(batch_size)
print(len(dados_treino1))    

235


## Estruturando os Dados:

Primeramente vou carregar os dados do dataset que baixei e transformá-los em Tensores para facilitar o cálculo. 

In [53]:

print(len(train_iter))   
print(len(test_iter))

235
40


In [26]:
print(len(dados_treino))
print(len(dados_teste))

60000
10000


In [27]:
numpy_array = dados_treino.to_numpy()
tensor = torch.from_numpy(numpy_array)

In [28]:
X = torch.zeros((60000, 28, 28))
Y = torch.zeros(60000, dtype=torch.int32)
for i in range(60000):
    Y[i] = tensor[i][0]
    X[i] = torch.reshape(tensor[i][1:], (28, 28))

In [29]:
numpy_array = dados_teste.to_numpy()
tensor = torch.from_numpy(numpy_array)

X_teste = torch.zeros((10000, 28, 28))
Y_teste = torch.zeros(10000, dtype=torch.int32)

for i in range(10000):
    Y_teste[i] = tensor[i][0]
    X_teste[i] = torch.reshape(tensor[i][1:], (28, 28))


In [30]:
## Normalizando os dados:
X = X.float() / 255

cont = torch.zeros(10) 
for i in range(len(Y)):
    cont[Y[i]] += 1

print(cont)


tensor([6000., 6000., 6000., 6000., 6000., 6000., 6000., 6000., 6000., 6000.])


## Vetores de Peso:

Vamos dar início aos hiperparâmetros iniciais, teremos 3 camadas nessa rede, a primeira que recebe 784 parâmetros da entrada (a imagem) e gera 256 saídas diferentes. A segunda camada que recebe 256 parâmetros e devolve 100. E por fim, a terceira camada que recebe 100 e gera 10. 

In [31]:
tamanho_entrada = 784
tamanho_saida = 10
numero_camadas_escondidas = 100

W1 = nn.Parameter(torch.randn(784, 256, requires_grad=True) * 0.01)
b1 = nn.Parameter(torch.zeros(256, requires_grad=True))
W2 = nn.Parameter(torch.randn(256, 100, requires_grad=True) * 0.01)
b2 = nn.Parameter(torch.zeros(100, requires_grad=True))
W3 = nn.Parameter(torch.randn(100, 10, requires_grad=True) * 0.01)
b3 = nn.Parameter(torch.zeros(10, requires_grad=True))

A função abaixo adiciona *não-linearidade* ao modelo, zerando todos valores negativos. 

In [32]:
def relu(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

## Modelo:

Para entender a função do modelo, primeiro vamos destacar as estruturas que temos:
+ $X \in \mathbb{R}^{1 \times 784}$ - A imagem após o *reshape*;
+ $W_1 \in \mathbb{R}^{784 \times 256}$ - O primeiro vetor de pesos;
+ $b_1 \in \mathbb{R}^{1 \times 256}$ - O primeiro bias;

Note que com a imagem com *reshape*, o número de colunas da imagem é igual ao número de linhas do $W_1$. Então, podemos fazer uma multiplicação de matrizes:

$$X \times W_1 = R_1 \in \mathbb{R}^{256}$$
$$R_1 = R_1 + b_1 $$

Depois disso, vamos lidar com o segundo vetor de pesos e o segundo bias:
+ $W_2 \in \mathbb{R}^{256 \times 100}$ - O segundo vetor de pesos;
+ $b_2 \in \mathbb{R}^{1 \times 100}$ - O segundo bias;

$$R_1 \times W_2 = R_2 \in \mathbb{R}^{10}$$
$$R_2 = R_2 + b_2$$

Por fim, vamos lidar com o terceiro vetor de pesos e o terceiro bias:
+ $W_3 \in \mathbb{R}^{100 \times 10}$ - O terceiro vetor de pesos;
+ $b_3 \in \mathbb{R}^{1 \times 10}$ - O terceiro bias;

$$R_2 \times W_3 = R_3 \in \mathbb{R}^{10}$$
$$R_3 = R_3 + b_3$$

O vetor final, será interpretado como a probabilidade da imagem pertencer a alguma das classes. 

In [33]:
def rede(X):
    X = X.reshape(-1, 784)
    H = relu(X @ W1 + b1)
    H2 = relu(H @ W2 + b2)
    return H2 @ W3 + b3

## Função de Perda:

In [34]:
perda = nn.CrossEntropyLoss()

In [35]:
class Accumulator: 
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    

def acuracia(y_hat, y):  
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

def avaliacao_acuracia(dados_teste, rotulos_teste):  
    metric = Accumulator(2)  
    for i in range(len(dados_teste)):
        metric.add(acuracia(rede(dados_teste[i]), rotulos_teste[i]), rotulos_teste[i].numel())
    return metric[0] / metric[1]

## Função de Treino:

In [36]:
def treino_epoca(rede, dados_treino, rotulos_treino, perda, updater):
    for i in range(len(dados_treino)):       
        Y_hat = rede(dados_treino[i])
        l = perda(Y_hat, rotulos_treino[i])

        updater.zero_grad()
        l.backward()
        updater.step()
        
    return metric[0] / metric[2], metric[1] / metric[2]

In [37]:
def treino(rede, dados_treino, rotulos_treino, dados_teste, perda, numero_epocas, updater):
    loss_list = []
    acc_list = []
    test_acc = []

    for i in range(numero_epocas):
        metrics = treino_epoca(rede, dados_treino, rotulos_treino, perda, updater)
        loss_list.append(metrics[0])
        acc_list.append(metrics[1])
        aux = avaliacao_acuracia(dados_teste)
        test_acc.append(aux)

    return loss_list, acc_list, test_acc

In [47]:
num_epocas = 10
lr = 0.1
params = [W1, b1, W2, b2, W3, b3]
updater = torch.optim.SGD(params, lr)
Y = Y.unsqueeze(1)
Y = Y.long()


loss_list, acc_list, test_acc = treino(rede, X, Y, X_teste, perda, num_epocas, updater)



RuntimeError: 0D or 1D target tensor expected, multi-target not supported